In [1]:
import numpy as np
import librosa as lb
import os
import os.path
import subprocess
import pathlib
from pathlib import Path
import random
import soundfile as sf
import pandas as pd
import glob
import pickle
import multiprocessing

In [2]:
FEATURES_ROOT = Path('/home/cchang/ttmp/features')
train_files = Path('cfg_files/filelist.train.txt')
test_files = Path('cfg_files/filelist.test.txt')

## Modify Data (Partial Matching)

### Functions

In [3]:
def modifyDataset(origdata, L, outdir, audiodir = 'wav_22050_mono'):
    '''
    Generates modified dataset with repeated material inserted
    
    origdata: the original data directory filepath
    L: the duration of the inserted fragment in seconds
    outdir: the desired output file directory (Path object)    
    '''
    random.seed(0)  # for reproducible results
    replacementTS = []
    copyFileStructure(origdata, outdir)  # copy directory structure

    # Now we need to get all the wav files we want to modify
    origWav = [f for f in glob.glob(origdata.as_posix() + "/" + audiodir + '/**/*.wav', recursive=True)]
    
    # structural exceptions
    file1 = 'Chopin_Op068No3_Koczalski-1948_pid9140-05.wav'
    file2 = 'Chopin_Op017No4_Ginzburg-1957_pid9156-10.wav'
    
    for wavFile in origWav:
        # The beat annotation files of this are outside the original Chopin Mazurka folder (structure exception)
        if os.path.basename(wavFile) == file1 or os.path.basename(wavFile) == file2:
            continue
            
        # check that the filler audio is not the same Mazurka as the original audio
        currentMazurka = Path(wavFile).parent
        otherMazurka = random.choice(os.listdir(origdata / audiodir))
        while otherMazurka == currentMazurka or otherMazurka[0] == ".":
            otherMazurka = random.choice(os.listdir(origdata / audiodir))
        otherMazurka = Path(wavFile).parents[1] / otherMazurka
        
        # check that the audio piece is not one of the exceptions
        fillerAudio = random.choice(os.listdir(otherMazurka))
    
        while fillerAudio == file1 or fillerAudio == file2 or fillerAudio[0] == ".":
            fillerAudio = random.choice(os.listdir(otherMazurka))
        fillerAudio = otherMazurka / fillerAudio
        
        # modify the audio and save it
        replacementTS.append(modifyData(Path(wavFile), fillerAudio, L, outdir))
        
           
    # Save timestamps into csv file
    replacementTS_np = np.array(replacementTS)
    pd.DataFrame(replacementTS_np).to_csv(outdir / 'replacement_ts.csv', index=None, header=['name', 'beginning replacement end (idx)', 'middle replacement start (idx)', 'middle replacement end (idx)', 'ending replacement start (idx)'])
    
    return

In [4]:
def copyFileStructure(origdir, outdir):
    '''copy the directory structure of the original data'''
    outdir.mkdir(parents=True, exist_ok=True)
    
    # Get subdirectories in original folder
    origDirList = [f for f in glob.glob(origdir.as_posix() + '/**/', recursive=True)]
    
    # Create subdirectories in output directory
    for origSubDir in origDirList:
        subDirPath = Path(origSubDir)
        index = subDirPath.parts.index(origdir.name)
        newSubDirPath = outdir.joinpath(*subDirPath.parts[index+1:])
        newSubDirPath.mkdir(parents=True, exist_ok=True)

In [5]:
def modifyData(wavFile, fillerWavFile, L, outdir):
    # Load in original wav
    name = os.path.basename(wavFile)
    origSound, sr = lb.core.load(wavFile)
    L_total = origSound.shape[0]  # total length of recording

    # replace the first L_start sec of the audio recording
    L_start = random.randint(0, 10*sr)
    filler_start, _ = lb.core.load(fillerWavFile, sr=sr, offset=0, duration=L_start/sr)
    np.put(origSound, np.arange(L_start), filler_start)

    # replace the last L_end sec of the audio recording
    L_end = random.randint(0, 10*sr)
    filler_end, _ = lb.core.load(fillerWavFile, sr=sr, offset=(L_total-L_end)/sr, duration=None)
    np.put(origSound, np.arange(L_total - L_end, L_total), filler_end)
    
    # replace a middle section of the audio recording
    if L_start + 10*sr > L_total - L_end - L*sr - 10*sr:
        print("No valid middle replacement for %s with duration %s, L %s, L_start %s, and L_end %s" % (name, L_total/sr, L, L_start/sr, L_end/sr))
        t_selected = -1
    else:
        t_selected = random.randint(L_start + 10*sr, L_total - L_end - L*sr - 10*sr)
        filler_mid, _ = lb.core.load(fillerWavFile, sr=sr, offset=t_selected/sr, duration = L)
        np.put(origSound, np.arange(t_selected, t_selected+L*sr), filler_mid)

    # Save modified audio file
#     oldPath = Path(wavFile)
    
    newPath = outdir.joinpath(*wavFile.parts[-3:])
    sf.write(newPath, origSound, 22050)
    
    modInfo = [name, L_start, t_selected, t_selected+L*sr, L_total - L_end]
    return modInfo

### Run Modification

In [6]:
# suppress warnings for PySoundFile
import warnings
warnings.filterwarnings("ignore")

In [7]:
origdata = Path('/home/cchang/ttmp/raw_data/Chopin_Mazurkas')
durations = [5, 10, 15, 20, 25, 30]

for L in durations:
    outdir = Path('/home/cchang/ttmp/raw_data/partial_match/Chopin_Mazurkas_partial_match_%ss' % L)
    modifyDataset(origdata, L, outdir)

## Create Features

In [8]:
def compute_chroma_single(infile, outfile, sr = 22050, hop_length=512):
    y, sr = lb.core.load(infile, sr = sr)
    #F = lb.feature.chroma_cens(y, sr=sr, hop_length=hop_length)
    F = lb.feature.chroma_cqt(y, sr=sr, hop_length=hop_length, norm=2)
    np.save(outfile, F)
    return

In [9]:
def compute_chroma_batch(audio_root, filelist, outdir, n_cores):
    # prep inputs for parallelization
    inputs = []
    with open(filelist, 'r') as f:
        for line in f:
            relpath = line.strip()
            reldir, fileid = os.path.split(relpath)
            featdir = outdir / reldir
            featdir.mkdir(parents=True, exist_ok=True)
            featfile = (featdir / fileid).with_suffix('.npy')
            audiofile = (audio_root / relpath).with_suffix('.wav')
            if os.path.exists(featfile):
                print(f"Skipping {featfile}")
            else:
                inputs.append((audiofile, featfile))

    # process files in parallel
    pool = multiprocessing.Pool(processes = n_cores)
    pool.starmap(compute_chroma_single, inputs)
    
    return

In [ ]:
# compute features on the clean audio

rootdir = '/home/cchang/ttmp/raw_data/Chopin_Mazurkas/'
ANNOTATIONS_ROOT = Path(rootdir + 'annotations_beat')
AUDIO_ROOT = Path(rootdir + 'wav_22050_mono')

FEATS_CLEAN_DIR = FEATURES_ROOT / 'clean'
compute_chroma_batch(AUDIO_ROOT, train_files, FEATS_CLEAN_DIR, 24)
compute_chroma_batch(AUDIO_ROOT, test_files, FEATS_CLEAN_DIR, 24)

In [ ]:
# compute features for partial modification (5, 10, 15, 20, 25, and 30 sec)
dirs = ['partial_match_5s', 'partial_match_10s', 'partial_match_15s', 'partial_match_20s', 'partial_match_25s', 'partial_match_30s']
AUDIO_ROOT = '/home/cchang/ttmp/raw_data/partial_match/'

for x in dirs:
    audioroot = Path(AUDIO_ROOT + "Chopin_Mazurkas_" + x + '/wav_22050_mono')
    compute_chroma_batch(audioroot, train_files, FEATURES_ROOT / 'partial_match' / x, 24)
    compute_chroma_batch(audioroot, test_files, FEATURES_ROOT / 'partial_match' / x, 24)